# Pruning LeNet-5 Model
This notebook shows how to reduce the size of a model by pruning its parameters. It assumes 
that a trained LeNet-5 model already exists in the ```Models``` directory. You can use the 
notebook [Handwritten Digit recognition (LeNet-5/MNIST)](LeNet5-MNIST.ipynb) to create and train a LeNet-5 model.

If you want to prune a Low-Rank model, you can use [this](LeNet5-MNIST-Reduce.ipynb) notebook
to reduce the number of parameters in ```LeNet-5```.

## Load and evaluate the original pretrained model

In [10]:
from fireball import Model
from fireball.datasets.mnist import MnistDSet

testDs = MnistDSet.makeDatasets('test', batchSize=128)

# orgFileName = "Models/LeNet5.fbm"   # original model
# orgFileName = "Models/LeNet5R.fbm"  # Reduced
orgFileName = "Models/LeNet5RR.fbm" # Reduced - Retrained

model = Model.makeFromFile(orgFileName, testDs=testDs, gpus='0')   
model.initSession()
model.printLayersInfo()
results = model.evaluate()



Reading from "Models/LeNet5RR.fbm" ... Done.
Creating the fireball model "LeNet-5" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 28x28x1      28 28 1       None                      0          
L1_CONV          28 28 1       KSP: 5 1 s               14 14 6       ReLU     MP(KSP):2 2 v    156        
L2_CONV          14 14 6       KSP: 5 1 v, LR8          5 5 16        ReLU     MP(KSP):2 2 v    1,344      
L3_FC            5 5 16        LR8                      120           ReLU                      4,280      
L4_FC            120           LR8                      84            ReLU                      1,716      
L5_FC            84                                     10            None                      850        
OUT_CLASS        10            10 classes

## Pruning the model
Here we prune the model using the ``pruneModel`` class method of the model.

In [11]:
prunedFileName = orgFileName.replace('.fbm', 'P.fbm')  # Append 'P' to the filename for "Pruned"
pResults = Model.pruneModel(orgFileName, prunedFileName, mseUb=.001, numWorkers=0)


Reading model parameters from "Models/LeNet5RR.fbm" ... Done.
Pruning 13 tensors ... 
   Pruning Parameters:
        mseUb ................ 0.001000
    Tensor 1 of 13 Shape: 5x5x1x6 ........... Done. 45 Pruned < 0.095147, MSE=0.00099, Reduced: 0.0%)
    Tensor 2 of 13 Shape: 6 ................. Ignored. (1-D Tensor)
    Tensor 3 of 13 Shape: 5x5x6x8 ........... Done. 466 Pruned < 0.087863, MSE=0.00100, Reduced: 0.0%)
    Tensor 4 of 13 Shape: 1x1x8x16 .......... Done. 22 Pruned < 0.114407, MSE=0.00095, Reduced: 0.0%)
    Tensor 5 of 13 Shape: 16 ................ Ignored. (1-D Tensor)
    Tensor 6 of 13 Shape: 400x8 ............. Done. 1367 Pruned < 0.091378, MSE=0.00100, Reduced: 0.0%)
    Tensor 7 of 13 Shape: 8x120 ............. Done. 323 Pruned < 0.089173, MSE=0.00099, Reduced: 0.0%)
    Tensor 8 of 13 Shape: 120 ............... Ignored. (1-D Tensor)
    Tensor 9 of 13 Shape: 120x8 ............. Done. 486 Pruned < 0.081078, MSE=0.00100, Reduced: 0.0%)
    Tensor 10 of 13 Shape: 8x

## Evaluate the pruned model
Compare the new number of parameters with the original 61,706. Let's see the impact of this reduction to the performance of the model.

In [12]:
model = Model.makeFromFile(prunedFileName, testDs=testDs, gpus='0')   
model.printLayersInfo()
model.initSession()
results = model.evaluate()


Reading from "Models/LeNet5RRP.fbm" ... Done.
Creating the fireball model "LeNet-5" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 28x28x1      28 28 1       None                      0          
L1_CONV          28 28 1       KSP: 5 1 s               14 14 6       ReLU     MP(KSP):2 2 v    111        
L2_CONV          14 14 6       KSP: 5 1 v, LR8          5 5 16        ReLU     MP(KSP):2 2 v    856        
L3_FC            5 5 16        LR8                      120           ReLU                      2,590      
L4_FC            120           LR8                      84            ReLU                      922        
L5_FC            84                                     10            None                      545        
OUT_CLASS        10            10 classe

## Re-training after pruning
Here we make a new model instance for re-training from the file created above. We then call the ```train``` method of the model to start the re-training.

After re-training, we run the ```evaluate``` function again to see how the re-training improved the performance
of the model.

The re-trained model is then saved to a file appending an 'R' letter (for Re-trained) to the end of the pruned model file name.

In [13]:
import os

trainDs, testDs = MnistDSet.makeDatasets('train,test', batchSize=128)
model = Model.makeFromFile(prunedFileName,
                           trainDs=trainDs, testDs=testDs, 
                           optimizer='Adam',
                           numEpochs=5,
                           learningRate=(0.01,0.0001),
                           gpus='0')
model.printLayersInfo()
model.initSession()
model.train()
results = model.evaluate()

retrainedFileName = prunedFileName.replace('.fbm', 'R.fbm')  # Append 'R' to the filename for "Retrained"
model.save(retrainedFileName)


Reading from "Models/LeNet5RRP.fbm" ... Done.
Creating the fireball model "LeNet-5" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 28x28x1      28 28 1       None                      0          
L1_CONV          28 28 1       KSP: 5 1 s               14 14 6       ReLU     MP(KSP):2 2 v    111        
L2_CONV          14 14 6       KSP: 5 1 v, LR8          5 5 16        ReLU     MP(KSP):2 2 v    856        
L3_FC            5 5 16        LR8                      120           ReLU                      2,590      
L4_FC            120           LR8                      84            ReLU                      922        
L5_FC            84                                     10            None                      545        
OUT_CLASS        10            10 classe

## Where do I go from here?

[Quantizing LeNet-5 Model](LeNet5-MNIST-Quantize.ipynb)

[Exporting LeNet-5 Model to ONNX](LeNet5-MNIST-ONNX.ipynb)

[Exporting LeNet-5 Model to TensorFlow](LeNet5-MNIST-TF.ipynb)

[Hand-written Didgit Recognition as a Regression problem](Regression.ipynb)
________________

[Fireball Playgrounds](../Contents.ipynb)

[Handwritten Digit Recognition (LeNet-5/MNIST)](LeNet5-MNIST.ipynb)

[Reducing number of parameters of LeNet-5 Model](LeNet5-MNIST-Reduce.ipynb)

